In [2]:
import requests
from bs4 import BeautifulSoup
import datetime

In [3]:
# Revisions API -- Documented at https://www.mediawiki.org/wiki/API:Revisions

S = requests.Session()

URL = "https://wikipedia.org/w/api.php"

In [4]:
start_time = datetime.datetime(2014,1,1)
parameters = {
    "action": "query",
    "prop": "revisions",
    "titles": "Barack_Obama",
    "rvprop": "ids|timestamp|comment",
    "rvslots": "main",
    "rvstart": start_time.isoformat(),
    "rvlimit": "5",
    "formatversion" : "2",
    "format": "json"
}

response = S.get(url=URL, params=parameters)


In [5]:
data = response.json()
display(data)

{'continue': {'rvcontinue': '20131225201818|587661731', 'continue': '||'},
 'query': {'normalized': [{'fromencoded': False,
    'from': 'Barack_Obama',
    'to': 'Barack Obama'}],
  'pages': [{'pageid': 534366,
    'ns': 0,
    'title': 'Barack Obama',
    'revisions': [{'revid': 588515537,
      'parentid': 588513334,
      'timestamp': '2013-12-31T12:30:50Z',
      'comment': 'Robot: [[yo:Barack Obama]] is a good article'},
     {'revid': 588513334,
      'parentid': 588233048,
      'timestamp': '2013-12-31T12:10:21Z',
      'comment': 'Robot: [[th:บารัค โอบามา]] is a good article'},
     {'revid': 588233048,
      'parentid': 588171071,
      'timestamp': '2013-12-29T17:27:47Z',
      'comment': 'Copyedit'},
     {'revid': 588171071,
      'parentid': 587936255,
      'timestamp': '2013-12-29T05:32:02Z',
      'comment': 'Changed visibility of the article feedback tool on "[[Barack Obama]]" ([Visibility=Enable for all users] (indefinite))'},
     {'revid': 587936255,
      'parenti

In [15]:
# REST API -- Documented 
parameters = {
        "oldid" : data['query']['pages'][0]['revisions'][0]['revid']
}
display(parameters)

{'oldid': 588515537}

In [16]:
INDEX_URL = 'https://en.wikipedia.org/w/index.php'
resp = S.get(INDEX_URL, params=parameters)

In [17]:
resp.text

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Barack Obama - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"586fa675-4597-4e2a-a5b3-0d23eb254378","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Barack_Obama","wgTitle":"Barack Obama","wgCurRevisionId":1019132373,"wgRevisionId":588515537,"wgArticleId":534366,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CS1 errors: unsupported parameter","CS1 maint: discouraged parameter","CS1 maint: multiple names: authors list","Pages containing links to subscription-only content",

In [12]:
soup = BeautifulSoup(resp.text)
# document_text = " ".join(t.text for t in soup.find_all('p'))
for div in soup.find_all("div", {'class':'mw-revision warningbox'}): 
    div.decompose()



In [13]:
document_text = " ".join(t.text for t in soup.find_all('p')[:25]) 

In [14]:
document_text

'This is the HTML representation of the JSON format. HTML is good for debugging, but is unsuitable for application use.\n Specify the format parameter to change the output format. To see the non-HTML representation of the JSON format, set format=json.\n See the complete documentation, or the API help for more information.\n'

In [35]:
API_URL = "https://wikipedia.org/w/api.php"
def get_wikipedia_revision(title, date, diagnostic=False):
    session = requests.Session()
    parameters = {
        "action": "query",
        "prop": "revisions",
        "titles": title,
        "rvprop": "ids|timestamp|comment",
        "rvslots": "main",
        "rvstart": date.isoformat(),
        "rvlimit": "1",
        "formatversion" : "2",
        "format": "json"
    }
    
    revision_response = session.get(url=API_URL, params=parameters)
    revision_response.raise_for_status()
    
    # revision data succeeded
    revision_data = revision_response.json()
    try:
        parameters = {
            "title" : title,
            "oldid" : revision_data['query']['pages'][0]['revisions'][0]['revid']
        }
    except (IndexError,KeyError) as e:
        if not diagnostic:
            return None, None
        else:
            raise e
    
    page_response = session.get(url=API_URL, params=parameters)
    
    return page_response.text, page_response.url
    
    
    
        
    

In [37]:
text, url = get_wikipedia_revision('fjdkslf', datetime.datetime.now())

In [34]:
text

'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>MediaWiki API help - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!0,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"b21a993a-8618-457a-9ba1-425ad61f8297","wgCSPNonce":!1,"wgCanonicalNamespace":"Special","wgCanonicalSpecialPageName":"ApiHelp","wgNamespaceNumber":-1,"wgPageName":"Special:ApiHelp","wgTitle":"ApiHelp","wgCurRevisionId":0,"wgRevisionId":0,"wgArticleId":0,"wgIsArticle":!1,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRelevantPageName":"Special:ApiHelp","wgRelevantArticleId":0,"wgIsProbablyEditable":!1,"wgRelevantPageIsP

In [31]:
def get_paragraphs(text):
    soup = BeautifulSoup(text)
# document_text = " ".join(t.text for t in soup.find_all('p'))
    for div in soup.find_all("div", {'class':'mw-revision warningbox'}): 
        div.decompose()
    document_text = " ".join(t.text for t in soup.find_all('p')[:25]) 
